In [2]:
%config ZMQInteractiveShell.ast_node_interactivity='all'
from pygoose import *
from sklearn.feature_extraction.text import TfidfVectorizer
import math

In [45]:
project = kg.Project.discover()
feature_list_id = 'wordmatchshare'

In [4]:
tokens_train = kg.io.load(project.preprocessed_data_dir + 'tokens_lowercase_spellcheck_no_stopwords_train.pickle')
tokens_test = kg.io.load(project.preprocessed_data_dir + 'tokens_lowercase_spellcheck_no_stopwords_test.pickle')

In [5]:
tokens = tokens_train+tokens_test

In [6]:
documents=[]
for pair in tokens:
    q1=[' '.join(pair[0])]
    q2=[' '.join(pair[1])]
    q=[q1,q2]
    documents.append(q)

# all_questions_flat = np.array(tokens).ravel()
# documents = list(set(' '.join(question) for question in all_questions_flat))
# del all_questions_flat
# documents

In [33]:
def get_tfidf_wordMatchShare(pair):#计算所有词的idf
    idf={}
    q_set = set()
    for question in pair:
        q1 = str(question[0][0])
        if q1 not in q_set:
            q_set.add(q1)
            words = q1.strip('.').split()
            for word in words:
                idf[word] = idf.get(word, 0) +1
        q2 = str(question[1][0])
        if q2 not in q_set:
            q_set.add(q2)
            words = q2.strip('.').split()
            for word in words:
                idf[word] = idf.get(word, 0)+1
    num_docs = len(pair)
    for word in idf:
        idf[word] = math.log(num_docs / (idf[word]+1.)) / math.log(2.)
    return idf

In [34]:
idf = get_tfidf_wordMatchShare(documents)

In [38]:
def get_wordMatchShare(question):
    q1words={}#q1中每个单词出现的次数
    q2words={}
    q1 = question[0]
    for word in q1:
        q1words[word] = q1words.get(word,0)+1
    q2 = question[1]
    for word in q2:
        q2words[word] = q2words.get(word,0)+1
    n_shared_word_in_q1 = sum([q1words[w] for w in q1words if w in q2words]) #q1和q2中共同出现的单词的次数
    n_shared_word_in_q2 = sum([q2words[w] for w in q2words if w in q1words])
    n_tol = sum(q1words.values())+sum(q2words.values())#q1和q2中所有单词出现的次数
    sum_shared_word_in_q1 = sum([q1words[w]*idf.get(w, 0) for w in q1words if w in q2words]) #share word的tfidf
    sum_shared_word_in_q2 = sum([q2words[w]*idf.get(w, 0) for w in q2words if w in q1words])
    sum_tol = sum(q1words[w]*idf.get(w,0) for w in q1words) +sum(q2words[w]*idf.get(w,0) for w in q2words)
    
    if 1e-6>n_tol:
        n_shared_word = 0.
    else:
        n_shared_word = 1.0*(n_shared_word_in_q1 + n_shared_word_in_q2)/n_tol
    if 1e-6>sum_tol:
        sum_shared_word = 0.
    else:
        sum_shared_word = 1.0*(sum_shared_word_in_q1+sum_shared_word_in_q2) /sum_tol
    return [n_shared_word, sum_shared_word]

In [39]:
features = kg.jobs.map_batch_parallel(
    tokens,
    item_mapper=get_wordMatchShare,
    batch_size=1000,
)


Batches:  10%|▉         | 274/2751 [02:30<23:26,  1.76it/s]


Batches:  19%|█▉        | 534/2751 [05:02<20:51,  1.77it/s]


Batches:  29%|██▉       | 794/2751 [07:29<18:45,  1.74it/s]


Batches:  38%|███▊      | 1053/2751 [09:53<15:42,  1.80it/s]


Batches:  48%|████▊     | 1309/2751 [12:21<13:49,  1.74it/s]


...

Batches:  66%|██████▌   | 1821/2751 [17:14<08:38,  1.79it/s]


Batches:  75%|███████▌  | 2077/2751 [19:38<06:17,  1.79it/s]


Batches:  85%|████████▍ | 2333/2751 [22:03<03:52,  1.80it/s]


Batches:  94%|█████████▍| 2589/2751 [24:26<01:30,  1.79it/s]


Batches: 100%|██████████| 2751/2751 [25:57<00:00,  1.80it/s]


In [41]:
features_train = np.array(features[:len(tokens_train)], dtype='float64')
features_test = np.array(features[len(tokens_train):], dtype='float64')

In [46]:
feature_names = [
    'wordmatchshare_percent',
    'wordmatchshare_tfidf'
]
project.save_features(features_train, features_test, feature_names, feature_list_id)